# Setup
Update path, load assemblies and import modules

In [2]:
import os
import sys
sys.path.extend([
    '../.venv/Lib',
    '../.venv/Lib/site-packages',
    '../LibSLH/bin/Debug/netstandard2.0'])
import clr
sys.path.append('')

clr.AddReference('System.Drawing')
clr.AddReference('LibreMetaverse')
clr.AddReference('LibSLH')

# Commonly used
from OpenMetaverse import *
from LibSLH import *

# Important imports
from slhstudio import *
from helpers import *

import System.Drawing.Imaging
import math
import asyncio
from IPython.display import display_html, clear_output

# Start the Client

In [3]:
client = Client()
client.start()

# Main event loop override
Make the client rotate in place by overriding the main update function. This ensures that world objects will begin to load for the client.

In [ ]:
def spin_client(client):
    while True:
        for i in range(16):
            theta = math.pi * i / 16
            direction = Vector3(math.cos(theta), math.sin(theta), 0.0)
            # Oof gamer https://github.com/pythonnet/pythonnet/issues/906
            rotation = Vector3.RotationBetween(Vector3.UnitX, Vector3.Normalize(direction))
            target = Vector3.Add(client.Self.SimPosition, direction)  
            client.Self.Movement.Camera.LookAt(client.Self.SimPosition, target);
            client.Self.Movement.SendUpdate()
            yield True

spin = spin_client(client)
async def my_update():
    next(spin)
    await asyncio.sleep(0.1)

client.update = my_update

# Examples

### Say a message in local chat

In [ ]:
def say(obj):
    client.Self.Chat(str(obj), 0, ChatType.Normal)

In [ ]:
say('hey lol')

### Misc. examples

In [ ]:
{str(e.Key): str(e.Value) for e in client.Network.CurrentSim.AvatarPositions.Copy()}

In [ ]:
{str(e.Key): str(e.Value) for e in client.Friends.FriendList.Copy()}

In [ ]:
def debug_point_at(sender, args):
    pt = str(args.PointType)
    id = str(args.TargetID)
    pos = str(args.TargetPosition)
    message = f'{pt}, {id}, {pos}'
    client.Self.Chat(message, 0, ChatType.Normal)
remove_all_event_handlers(client.Avatars, 'ViewerEffectPointAt')
client.Avatars.ViewerEffectPointAt += debug_point_at

### Debug Textures!

In [ ]:
def get_prim_texture_info(prim):
    yield prim.Textures.DefaultTexture
    yield from filter(lambda t: t, prim.Textures.FaceTextures)

def display_multi_image(paths, max_width=256):
    tags = [f'<img src="{path}" style="display: inline-block; max-width: {max_width}px">' for path in paths]
    display_html('\n'.join(tags), raw=True)

def get_image_path(uuid):
    path = os.path.join('img_cache', str(uuid) + '.png')
    if not os.path.exists(path):
        client.GetTextureByUUID(uuid).Save(path)
    return path

def debug_textures_on_selection(sender, args):
    if args.PointType == PointAtType.Select:
        local_id = client.GetPrimLocalId(args.TargetID)
        parent_id = client.GetParentLocalId(local_id)
        parent_id = parent_id or local_id
        link_set = list(client.GetLinkSetLocalIds(parent_id))
        prims = [client.Objects.GetPrimitive(args.Simulator, System.UInt32(id), UUID.Zero, False) for id in link_set]
        tex_info = [info for prim in prims for info in get_prim_texture_info(prim)]
        tex_uuids = list({info.TextureID for info in tex_info})
        clear_output()
        print("Loading images, please wait...")
        file_paths = [get_image_path(uuid) for uuid in tex_uuids]
        clear_output()
        display_multi_image(file_paths, 64)
remove_all_event_handlers(client.Avatars, 'ViewerEffectPointAt')
client.Avatars.ViewerEffectPointAt += debug_textures_on_selection

# Stop the client

In [7]:
client.stop()